In [1]:
#Run cell to mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# install package to have access to custom functions
%pip install /content/drive/MyDrive/EMG_gestures/ --use-feature=in-tree-build

Processing ./drive/MyDrive/EMG_gestures
  Created wheel for EMG-gestures: filename=EMG_gestures-0.1.0-py3-none-any.whl size=35784 sha256=cfb4bd7a64f8ba7295018f01dc8df5073c3a66e2ab8eec8afa93be0e8f3acfb3
  Stored in directory: /tmp/pip-ephem-wheel-cache-n1y6e4fe/wheels/a2/b7/61/2147fa082a9e51bef5dcc38dd3f0898fe0554d62203c0e383e
Successfully built EMG-gestures


In [3]:
#import necessary packages

#our workhorses
import numpy as np
import pandas as pd
import scipy

#to visualize
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
#style params for figures
sns.set(font_scale = 2)
plt.style.use('seaborn-white')
plt.rc("axes", labelweight="bold")
from IPython.display import display, HTML

#to load files
import os
import sys
import h5py
import pickle

#import cusotm functions
from EMG_gestures.utils import *
from EMG_gestures.analysis import log_reg_xsubject_transform_module_train_frac_subjects, log_reg_xsubject_transform_module_train_all_subjects


In [4]:
#define where the data files are located
data_folder = '/content/drive/MyDrive/EMG_gestures/EMG_data/'

nsubjects = 36

#randomly-selected subjects to use as hold-out test data 
test_subjects = [17, 23,  7,  8,  3]

# User-defined parameters
lo_freq = 20 #lower bound of bandpass filter
hi_freq = 450 #upper bound of bandpass filter

win_size = 100 #define window size over which to compute time-domain features
step = win_size #keeping this parameter in case we want to re-run later with some overlap



In [5]:
#intialize empty lists
feature_matrix_all = np.empty((0,0))
target_labels_all = np.empty((0,))
window_tstamps_all = np.empty((0,))
block_labels_all  = np.empty((0,))
series_labels_all  = np.empty((0,))
subject_id_all = np.empty((0,))
block_count = 0

for subject_id in range(1,nsubjects+1):
    if subject_id not in test_subjects:
        subject_folder = os.path.join(data_folder,'%02d'%(subject_id))
        print('=======================')
        print(subject_folder)

        # Process data and get features 
        #get features across segments and corresponding info
        feature_matrix, target_labels, window_tstamps, \
        block_labels, series_labels = get_subject_data_for_classification(subject_folder, lo_freq, hi_freq, \
                                                                        win_size, step)

        #prevent repeat of block labels by increasing block count
        block_labels = block_labels+block_count
        block_count = np.max([block_count, np.max(block_labels)])


        # concatenate lists
        feature_matrix_all = np.vstack((feature_matrix_all,feature_matrix)) if feature_matrix_all.size else feature_matrix
        target_labels_all = np.hstack((target_labels_all,target_labels))
        window_tstamps_all = np.hstack((window_tstamps_all,window_tstamps))
        block_labels_all = np.hstack((block_labels_all,block_labels))
        series_labels_all = np.hstack((series_labels_all,series_labels))
        subject_id_all = np.hstack((subject_id_all,np.ones((block_labels.size))*subject_id))
        

/content/drive/MyDrive/EMG_gestures/EMG_data/01
/content/drive/MyDrive/EMG_gestures/EMG_data/02
/content/drive/MyDrive/EMG_gestures/EMG_data/04
/content/drive/MyDrive/EMG_gestures/EMG_data/05
/content/drive/MyDrive/EMG_gestures/EMG_data/06
/content/drive/MyDrive/EMG_gestures/EMG_data/09
/content/drive/MyDrive/EMG_gestures/EMG_data/10
/content/drive/MyDrive/EMG_gestures/EMG_data/11
/content/drive/MyDrive/EMG_gestures/EMG_data/12
/content/drive/MyDrive/EMG_gestures/EMG_data/13
/content/drive/MyDrive/EMG_gestures/EMG_data/14
/content/drive/MyDrive/EMG_gestures/EMG_data/15
/content/drive/MyDrive/EMG_gestures/EMG_data/16
/content/drive/MyDrive/EMG_gestures/EMG_data/18
/content/drive/MyDrive/EMG_gestures/EMG_data/19
/content/drive/MyDrive/EMG_gestures/EMG_data/20
/content/drive/MyDrive/EMG_gestures/EMG_data/21
/content/drive/MyDrive/EMG_gestures/EMG_data/22
/content/drive/MyDrive/EMG_gestures/EMG_data/24
/content/drive/MyDrive/EMG_gestures/EMG_data/25
/content/drive/MyDrive/EMG_gestures/EMG_

In [6]:
#define hyper params for each model
model_dict = {1:{'n_dense_pre':1, 'activation':'linear'},\
              2:{'n_dense_pre':1, 'activation':'tanh'},\
              3:{'n_dense_pre':1, 'activation':'relu'},\
              4:{'n_dense_pre':2, 'activation':'tanh'},\
              5:{'n_dense_pre':2, 'activation':'relu'},\
}

In [9]:
#network training args 
verbose = 0
epochs = 30
batch_size = 5

#validation scheme args
n_train_splits = 4
n_val_splits = 2
nreps = 10

#excluded labels
exclude = [0,7]
#performance metrics
score_list = ['f1','accuracy']


results_folder = '/content/drive/MyDrive/EMG_gestures/results_data/xsubject_transform_module/log_reg/'

for model_id in range(1,5+1):
    results_model_df = []
    np.random.seed(1)#to replicate results
    for rep in range(nreps):

        print('Model %i || Rep %02d'%(model_id, rep+1))
        print('----True Data----')
        rep_results_df = log_reg_xsubject_transform_module_train_frac_subjects(feature_matrix_all, target_labels_all, subject_id_all, block_labels_all,\
                                                        series_labels_all, exclude, model_dict[model_id],score_list,\
                                                        n_train_splits = n_train_splits,n_val_splits = n_val_splits,\
                                                        verbose = verbose, epochs = epochs, batch_size = batch_size,permute = False)
        #add details and concatenate dataframe
        rep_results_df['Shuffled'] = False
        rep_results_df['Rep'] =  rep+1
        rep_results_df['Model'] = model_id
        results_model_df.append(rep_results_df)

        print('Model %i || Rep %02d'%(model_id, rep+1))
        print('----Permuted Data----')
        rep_results_df = log_reg_xsubject_transform_module_train_frac_subjects(feature_matrix_all, target_labels_all, subject_id_all, block_labels_all,\
                                                        series_labels_all, exclude, model_dict[model_id],score_list,\
                                                        n_train_splits = n_train_splits,n_val_splits = n_val_splits,\
                                                        verbose = verbose, epochs = epochs, batch_size = batch_size, permute = True)
        # add details and concatenate dataframe
        rep_results_df['Shuffled'] = True
        rep_results_df['Rep'] =  rep+1
        rep_results_df['Model'] = model_id
        results_model_df.append(rep_results_df)

    results_model_df = pd.concat(results_model_df,axis = 0)
    #save results to file
    results_fn = 'model_%02d_results.h5'%(model_id)
    results_model_df.to_hdf(os.path.join(results_folder,results_fn), key='results_df', mode='w')
print('***Finished!**')

Streaming output truncated to the last 5000 lines.
Training: Subject 19 out of 23
Training Model
Evaluate Model on Trained Data
Training: Subject 20 out of 23
Training Model
Evaluate Model on Trained Data
Training: Subject 21 out of 23
Training Model
Evaluate Model on Trained Data
Training: Subject 22 out of 23
Training Model
Evaluate Model on Trained Data
Training: Subject 23 out of 23
Training Model
Evaluate Model on Trained Data
Validation: Subject 01 out of 08
Validation: Subject 02 out of 08
Validation: Subject 03 out of 08
Validation: Subject 04 out of 08
Validation: Subject 05 out of 08
Validation: Subject 06 out of 08
Validation: Subject 07 out of 08
Validation: Subject 08 out of 08
-------Split Count: 2-------
Training: Subject 01 out of 23
Training Model
Evaluate Model on Trained Data
Training: Subject 02 out of 23
Training Model
Evaluate Model on Trained Data
Training: Subject 03 out of 23
Training Model
Evaluate Model on Trained Data
Training: Subject 04 out of 23
Training 

In [10]:
results_folder = '/content/drive/MyDrive/EMG_gestures/results_data/xsubject_transform_module/log_reg/'
model_dir = '/content/drive/MyDrive/EMG_gestures/model_data/xsubject_transform_module/log_reg/'
figure_dir = '/content/drive/MyDrive/EMG_gestures/figures/training_history/xsubject_transform_module/log_reg/'

#network training args 
verbose = 0
epochs = 30
batch_size = 2
nreps = 10

#use first series to train (let it be input)
# train_idxs = np.where(series_labels_all==0)[0]
# test_idxs = np.where(series_labels_all==1)[0]

train_idxs = np.where(series_labels_all>=0)[0]
test_idxs = np.array([])
exclude = [0,7]

score_list = ['f1','accuracy']#performance metrics

model_id = 1
results_model_df = []
np.random.seed(1)
for rep in range(nreps):

    print('Model %i || Rep %02d'%(model_id, rep+1))
    print('----True Data----')
    figure_folder = os.path.join(figure_dir,'rep_%i'%(rep+1))
    if not os.path.isdir(figure_folder):
        os.makedirs(figure_folder)
    model_folder = os.path.join(model_dir,'rep_%i'%(rep+1))
    if not os.path.isdir(model_folder):
        os.makedirs(model_folder)

    results_df, scaler = log_reg_xsubject_transform_module_train_all_subjects(feature_matrix_all, target_labels_all, subject_id_all, block_labels_all,\
                                                            train_idxs, test_idxs, exclude, model_dict[model_id],score_list,\
                                                            figure_folder = figure_folder, model_folder = model_folder,\
                                                            verbose = verbose, epochs = epochs, batch_size = batch_size,\
                                                            permute = False)
    #fill in details and append to list
    results_df['Shuffled'] = False
    results_df['Rep'] = rep+1
    results_model_df.append(results_df)

    print('Model %i || Rep %02d'%(model_id, rep+1))
    print('----Permuted Data----')
    results_df, scaler = log_reg_xsubject_transform_module_train_all_subjects(feature_matrix_all, target_labels_all, subject_id_all, block_labels_all,\
                                                            train_idxs, test_idxs, exclude, model_dict[model_id],score_list,\
                                                            figure_folder = None, model_folder = None,\
                                                            verbose = verbose, epochs = epochs, batch_size = batch_size,\
                                                            permute = True)
    #fill in details and append to list
    results_df['Shuffled'] = True
    results_df['Rep'] = rep+1
    results_model_df.append(results_df)
results_model_df = pd.concat(results_model_df,axis = 0)

#save results to file
results_fn = 'train_model_transform_module_all_training_data_results.h5'
results_model_df.to_hdf(os.path.join(results_folder,results_fn), key='results_df', mode='w')

#save scaler
scaler_fn = 'trained_scaler_all_training_data.pkl'
with open(os.path.join(model_dir,scaler_fn), "wb") as output_file:
    pickle.dump(scaler, output_file)

Model 1 || Rep 01
----True Data----
Training: Subject 01 out of 31
Training Model
Evaluate Model on Trained Data
Training: Subject 02 out of 31
Training Model
Evaluate Model on Trained Data
Training: Subject 03 out of 31
Training Model
Evaluate Model on Trained Data
Training: Subject 04 out of 31
Training Model
Evaluate Model on Trained Data
Training: Subject 05 out of 31
Training Model
Evaluate Model on Trained Data
Training: Subject 06 out of 31
Training Model
Evaluate Model on Trained Data
Training: Subject 07 out of 31
Training Model
Evaluate Model on Trained Data
Training: Subject 08 out of 31
Training Model
Evaluate Model on Trained Data
Training: Subject 09 out of 31
Training Model
Evaluate Model on Trained Data
Training: Subject 10 out of 31
Training Model
Evaluate Model on Trained Data
Training: Subject 11 out of 31
Training Model
Evaluate Model on Trained Data
Training: Subject 12 out of 31
Training Model
Evaluate Model on Trained Data
Training: Subject 13 out of 31
Training 